# Importando as Bibliotecas e o DataFrame

In [ ]:
#importando bibliotecas
import pandas as pd
import numpy as np
from google.colab import data_table
data_table.enable_dataframe_formatter()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# England Premier League
df1 = pd.read_csv('https://www.football-data.co.uk/mmz4281/2122/E0.csv')
df2 = pd.read_csv('https://www.football-data.co.uk/mmz4281/2223/E0.csv')
df = [df1, df2]
df = pd.concat(df)
df = df[['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','B365H','B365D','B365A','B365>2.5','B365<2.5']]
df.columns = ['Date','Home','Away','FT_Goals_H','FT_Goals_A','FT_Result','FT_Odds_H','FT_Odds_D','FT_Odds_A','FT_Odds_Over25','FT_Odds_Under25'] 
# Ajustando o Índice do DataFrame
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)
df = df.dropna()

In [ ]:
df

In [ ]:
def Prob_Home():
    P_H = 1/df.FT_Odds_H
    return P_H

def Prob_Draw():
    P_D = 1/df.FT_Odds_D
    return P_D

def Prob_Away():
    P_A = 1/df.FT_Odds_A
    return P_A

In [ ]:
df['p_H'] = Prob_Home()
df['p_D'] = Prob_Draw()
df['p_A'] = Prob_Away()

In [ ]:
def Valor_do_Gol_H():
    VG_H = df.FT_Goals_H * df.p_A
    return VG_H 

def Valor_do_Gol_A():
    VG_A = df.FT_Goals_A * df.p_H
    return VG_A 

In [ ]:
df['Valor_do_Gol_H'] = Valor_do_Gol_H()
# Média
df['Media_Valor_do_Gol_H'] = df.groupby('Home')['Valor_do_Gol_H'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True).shift(1)
# Desvio Padrão
df['DP_Valor_do_Gol_H'] = df.groupby('Home')['Valor_do_Gol_H'].rolling(window=10, min_periods=1).std().reset_index(0,drop=True).shift(1)
# Coeficiente de Variação
df['CV_Media_Valor_do_Gol_H'] = df['DP_Valor_do_Gol_H'] / df['Media_Valor_do_Gol_H']

df['Valor_do_Gol_A'] = Valor_do_Gol_A()
# Média
df['Media_Valor_do_Gol_A'] = df.groupby('Away')['Valor_do_Gol_A'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True).shift(1)
# Desvio Padrão
df['DP_Valor_do_Gol_A'] = df.groupby('Away')['Valor_do_Gol_A'].rolling(window=10, min_periods=1).std().reset_index(0,drop=True).shift(1)
# Coeficiente de Variação
df['CV_Media_Valor_do_Gol_A'] = df['DP_Valor_do_Gol_A'] / df['Media_Valor_do_Gol_A']

# Over 2.5
df.loc[(df['FT_Goals_H'] + df['FT_Goals_A']) > 2, 'Over25'] = 1
df.loc[(df['FT_Goals_H'] + df['FT_Goals_A']) < 3, 'Over25'] = 0

df.replace(np.inf, np.nan, inplace=True)
df.dropna(inplace=True)

In [ ]:
df = df[['Date','Home','Away','FT_Goals_H','FT_Goals_A','FT_Result','p_H','p_D','p_A','FT_Odds_Over25','Valor_do_Gol_H','Media_Valor_do_Gol_H','CV_Media_Valor_do_Gol_H','Valor_do_Gol_A','Media_Valor_do_Gol_A','CV_Media_Valor_do_Gol_A','Over25']]
df = df.dropna()

In [ ]:
team = "Chelsea"
df0 = df.groupby(['Home']).get_group(team)
df0 = df0.tail(11)
df0

In [ ]:
# Selecionando o Filtro Estatítisco
flt = (df.Media_Valor_do_Gol_H > 0.1) & (df.Media_Valor_do_Gol_A > 0.1) & (df.CV_Media_Valor_do_Gol_H < 1) & (df.CV_Media_Valor_do_Gol_A < 1)
df2 = df[flt];
# Ajustando o Índice
df2.reset_index(inplace=True, drop=True)
df2.index = df2.index.set_names(['Nº'])
df2 = df2.rename(index=lambda x: x + 1)

Stake = 1
green = Stake * (df2.FT_Odds_Over25 - 1) 
red = -Stake

# Over 2.5 FT
df2.loc[((df2['Over25']) == 1), 'Profit_Over25'] = green
df2.loc[((df2['Over25']) == 0), 'Profit_Over25'] = red
df2['Profit_acu_Over25'] = df2.Profit_Over25.cumsum()
df2.Profit_acu_Over25.plot(title='Over 2.5 FT', xlabel='Quantidade de Apostas', ylabel='Stakes')
df2.Profit_acu_Over25.tail(1)

In [ ]:
ROI = df2.Profit_acu_Over25.tail(1)/len(df2)*100
ROI